# Práctica 1. Aprendizaje Automático

Authors: Carlos Iborra Llopis (100451170), Alejandra Galán Arrospide (100451273)


# 0. Table of contents

- [Práctica 1. Aprendizaje Automático](#práctica-1-aprendizaje-automático)
  - [0. Table of contents](#0-table-of-contents)
  - [1. Requirements](#1-requirements)
  - [2. Reading the datasets](#2-reading-the-datasets)


# 1. Requirements


In [ ]:
""" Importing necessary libraries """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno


# 2. Reading the datasets

Reading the datasets from the bz2 files, group 2.


In [ ]:
""" Reading the dataset """
disp_df = pd.read_csv("../data/disp_st2ns1.txt.bz2",
                      compression="bz2", index_col=0)
comp_df = pd.read_csv("../data/comp_st2ns1.txt.bz2",
                      compression="bz2", index_col=0)


# 3. EDA


**Key Concepts of Exploratory Data Analysis**

- **2 types of Data Analysis**
  - Confirmatory Data Analysis
  - Exploratory Data Analysis
- **4 Objectives of EDA**
  - Discover Patterns
  - Spot Anomalies
  - Frame Hypothesis
  - Check Assumptions
- **2 methods for exploration**
  - Univariate Analysis
  - Bivariate Analysis
- **Stuff done during EDA**
  - Trends
  - Distribution
  - Mean
  - Median
  - Outlier
  - Spread measurement (SD)
  - Correlations
  - Hypothesis testing
  - Visual Exploration


## 3.1. Dataset description
- **apcp_sfc**: 3-Hour accumulated precipitation at the surface (kg·m⁽⁻²⁾)
- **dlwrf_sfc**: Downward long-wave radiative flux average at the surface (W·m⁽⁻²⁾)
- **dswrf_sfc**: Downward short-wave radiative flux average at the surface (W·m⁽⁻²⁾)
- **pres_msl**: Air pressure at mean sea level (Pa)
- **pwat_eatm**: Precipitable Water over the entire depth of the atmosphere (kg·m⁽⁻²⁾)
- **spfh_2m**: Specific Humidity at 2 m above ground (kg·kg⁽⁻¹⁾)
- **tcdc_eatm**: Total cloud cover over the entire depth of the atmosphere (%)
- **tcolc_eatm**: Total column-integrated condensate over the entire atmos. (kg·m⁽⁻²⁾)
- **tmax_2m**: Maximum Temperature over the past 3 hours at 2 m above the ground (K)
- **tmin_2m**: Mininmum Temperature over the past 3 hours at 2 m above the ground (K)
- **tmp_2m**: Current temperature at 2 m above the ground (K)
- **tmp_sfc**: Temperature of the surface (K)
- **ulwrf_sfc**: Upward long-wave radiation at the surface (W·m⁽⁻²⁾)
- **ulwrf_tatm**: Upward long-wave radiation at the top of the atmosphere (W·m⁽⁻²⁾)
- **uswrf_sfc**: Upward short-wave radiation at the surface (W·m⁽⁻²⁾)


In [ ]:
disp_df.describe()


In [ ]:
disp_df.shape


In [ ]:
disp_df.head()


In [ ]:
result = disp_df.eq(0.0).sum()/len(disp_df)*100

# select only results over 50.0
result = result[result > 30.0]
result = result.sort_values(ascending=False)
result


In [ ]:
disp_df.isna().sum()


In [ ]:
disp_df.info()


## 3.2. Missing values

### Observations
The variables with most ammount of zeros are:
- **dswrf_s1_1**: Downward short-wave radiative flux average at the surface, at 12:00 UTC, normal to have a lot of zeros as it is not sunny at 12:00
- **uswrf_s1_1**: Upward short-wave radiation at the surface, at 12:00 UTC, normal to have a lot of zeros as it is not sunny at 12:00
- **apcp_s**: 3-Hour accumulated precipitation at the surface, as it is not raining every day, it is normal to have a lot of zeros
- **tcdc_ea**: Total cloud cover over the entire depth of the atmosphere, as it is not cloudy every day, it is normal to have a lot of zeros

First, lets start by assigning the zeros to NaNs. By doing this we can visualize the varibles that take more values other than zero.

In [ ]:
disp_df_nan = disp_df.replace(0.0, np.nan)

In [ ]:
# Sustitute 0.0 values with NaN and plot the name of the columns with missing values
msno.bar(disp_df_nan, labels=True, fontsize=10)
# Exporting image as png to ../data/img folder
plt.savefig("../data/img/missing_values.png")


In [ ]:
""" Plotting the missing values in a matrix """
msno.matrix(disp_df_nan, labels=True, fontsize=10, color=(.3, .3, .3))
# Exporting image as png to ../data/img folder
plt.savefig("../data/img/missing_values_matrix.png")


In [ ]:
msno.heatmap(disp_df_nan, fontsize=10)
# Exporting image as png to ../data/img folder
plt.savefig("../data/img/missing_values_heatmap.png")


In [ ]:
""" Plotting the dendrogram """
msno.dendrogram(disp_df_nan, fontsize=10)
# Exporting image as png to ../data/img folder
plt.savefig("../data/img/missing_values_dendrogram.png")